# Replication of Bansal et al. (2023)'s ITVReg [DEPRECATED]

Deprecated because the original code is a mess and it would be easier (and a better scientific validation)
to try to re-implement their method based on the math proposed in their paper. 

Found the original code (messy) at https://github.com/pbansal5/causal-recommendations. Going to try to clean up and replicate results. In particular the interventions are unclear both in their description in the paper and in the code, so some liberty may need to be taken there. 

In [1]:
# removing xclib requirement since it is a small library and does not support building on 
# new versions of python. Riddled with errors, wasn't able to patch all of them quickly. 

# nagame also removed, not even working in the OG work code lol

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import os
import transformers as ppb
from transformers import RobertaModel, AutoConfig
import warnings
from sentence_transformers import SentenceTransformer
from torch.utils.tensorboard import SummaryWriter
warnings.filterwarnings('ignore')
# from xclib.data.data_utils import read_sparse_file
import copy
from transformers import get_scheduler,AdamW
from transformers import AutoTokenizer
import nlpaug.augmenter.word as naw
import nlpaug
import nltk
# import xclib.evaluation.xc_metrics as xc_metrics
# import xclib.data.data_utils as data_utils
import scipy.sparse as sp
# from ngame.nns import exact_search


# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')

torch.set_default_device("mps")

/Users/danielfrees/miniconda3/envs/causalign/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
TOP_DIR = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))

In [4]:
import os
import gdown
import zipfile
import numpy as np
import json
import gzip

# Set the directory to store data (no need to set TOP_DIR)
data_dir = 'data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# Step 1: Download the dataset from Google Drive using the correct link
url = "https://drive.google.com/uc?id=1WuquxCAg8D4lKr-eZXPv4nNw2S2lm7_E"
output_zip = os.path.join(data_dir, 'LF-AmazonTitles-131K.raw.zip')
if not os.path.exists(output_zip):
    gdown.download(url, output_zip, quiet=False)

# Step 2: Extract the downloaded .zip file
with zipfile.ZipFile(output_zip, 'r') as zip_ref:
    zip_ref.extractall(data_dir)

# Step 3: Load the compressed JSON data from 'trn.json.gz', 'tst.json.gz', and 'lbl.json.gz'
def load_gz_json_line_by_line(file_path):
    data = []
    with gzip.open(file_path, 'rt', encoding='utf-8') as f:
        for line in f:
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError:
                print(f"Error decoding JSON on line: {line}")
    return data

# Paths for train, test, and label JSON data
unzipped_dir = "LF-Amazon-131K"
train_data_path = os.path.join(data_dir, unzipped_dir, 'trn.json.gz')
test_data_path = os.path.join(data_dir, unzipped_dir, 'tst.json.gz')
label_data_path = os.path.join(data_dir, unzipped_dir, 'lbl.json.gz')

# Load the datasets line by line
train_data = load_gz_json_line_by_line(train_data_path)
test_data = load_gz_json_line_by_line(test_data_path)
label_data = load_gz_json_line_by_line(label_data_path)

# Step 4: Optionally load the filter label files (if needed)
filter_train_path = os.path.join(data_dir, unzipped_dir, 'filter_labels_train.txt')
filter_test_path = os.path.join(data_dir, unzipped_dir, 'filter_labels_test.txt')

with open(filter_train_path, 'r') as f:
    filter_train = f.readlines()

with open(filter_test_path, 'r') as f:
    filter_test = f.readlines()

# Step 5: Print out some information about the loaded datasets
print(f"Loaded {len(train_data)} training records")
print(f"Loaded {len(test_data)} testing records")
print(f"Loaded {len(label_data)} labels")

# Optionally print a sample to see the structure
if train_data:
    print("Sample training record:", train_data[0])
if test_data:
    print("Sample test record:", test_data[0])
if label_data:
    print("Sample label:", label_data[0])

Loaded 294805 training records
Loaded 134835 testing records
Loaded 131073 labels
Sample training record: {'uid': 'B0007J0FJ0', 'title': 'Methodical Bible study: A new approach to hermeneutics', 'content': 'Inductive study compares related Bible texts in order to let the Bible interpret itself, rather than approaching Scripture with predetermined notions of what it will say.  Dr. Trainas Methodical Bible Study was not intended to be the last word in inductive Bible study; but since its first publication in 1952, it has become a foundational text in this field.  Christian colleges and seminaries have made it required reading for beginning Bible students, while many churches have used it for their lay Bible study groups.  Dr. Traina summarizes its success in this comment:  "If the truths of the Bible already resided in man, there would be no need for the Bible and this manual would be superfluous.  But the fact is the Bible is an objective body of literature which exists because man need

In [5]:
import numpy as np
import scipy.sparse as sp

# Example: Using the data from earlier

# Assume train_data, test_data, and label_data are already loaded from the JSON files
# Here we'll use the 'target_ind' and 'target_rel' fields to build sparse matrices

def create_sparse_matrix(data, num_rows, num_cols):
    """Create a sparse matrix from 'target_ind' and 'target_rel'."""
    row_indices = []
    col_indices = []
    values = []
    
    for row, record in enumerate(data):
        indices = record['target_ind']
        rels = record['target_rel']
        row_indices.extend([row] * len(indices))
        col_indices.extend(indices)
        values.extend(rels)
    
    # Create a sparse matrix in CSR format
    sparse_matrix = sp.csr_matrix((values, (row_indices, col_indices)), shape=(num_rows, num_cols))
    
    return sparse_matrix

# Get the dimensions
num_train_samples = len(train_data)
num_test_samples = len(test_data)
num_labels = len(label_data)  # This should be the number of columns (unique target indices)

# Create the sparse matrices
train_sparse_matrix = create_sparse_matrix(train_data, num_train_samples, num_labels)
test_sparse_matrix = create_sparse_matrix(test_data, num_test_samples, num_labels)

# Example: Print information about the created sparse matrix
print(f"Training sparse matrix shape: {train_sparse_matrix.shape}")
print(f"Testing sparse matrix shape: {test_sparse_matrix.shape}")

# If you want to see the non-zero elements count:
print(f"Training sparse matrix non-zero elements: {train_sparse_matrix.nnz}")
print(f"Testing sparse matrix non-zero elements: {test_sparse_matrix.nnz}")

Training sparse matrix shape: (294805, 131073)
Testing sparse matrix shape: (134835, 131073)
Training sparse matrix non-zero elements: 674996
Testing sparse matrix non-zero elements: 284746


In [6]:
class BertDataset(torch.utils.data.Dataset):

    def __init__(self, datadir,mapping, device,split):
        self.lbl_size = 131073
        self.datadir = datadir
        self.device=device
        self.size = 294805 if split =='trn' else 134835
        if split == 'trn':
            self.point_text_files = [x.strip() for x in open('%s/raw/trn_X.title.txt'%self.datadir).readlines()]
        else :
            self.point_text_files = [x.strip() for x in open('%s/raw/tst_X.title.txt'%self.datadir).readlines()]
        self.label_text_files = [x.strip() for x in open('%s/raw/Y.title.txt'%self.datadir).readlines()]
        self.mat_mapping = mapping
        self.mapping = mapping.nonzero()
        self.tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
        self.maxsize=32
        self.aug = naw.SynonymAug(aug_src='wordnet',aug_max=5)
        # aug = naw.WordEmbsAug(model_type='word2vec',model_path = './wiki-news-300d-1M.vec')
        temp = np.fromfile('%s/tst_filter_labels.txt'%self.datadir, sep=' ').astype(int)
        temp = temp.reshape(-1, 2).T
        self.tst_filter_mat = sp.coo_matrix((np.ones(temp.shape[1]), (temp[0], temp[1])), tst_X_Y.shape).tocsr()

    def __getitem__(self,index,augment=True):
        point_data,label_data = self.convert_joint(self.point_text_files[self.mapping[0][index]],self.label_text_files[self.mapping[1][index]],augment=augment)
#         point_data = self.convert(self.point_text_files[self.mapping[0][index]],augment=augment)
#         label_data = self.convert(self.label_text_files[self.mapping[1][index]],augment=augment)
        return (torch.Tensor(point_data['input_ids'][0]).to(device),
               torch.Tensor(label_data['input_ids'][0]).to(device),
               torch.Tensor(point_data['attention_mask'][0]).to(device),
               torch.Tensor(label_data['attention_mask'][0]).to(device))
    
    def convert_joint(self,textp,textl,augment=True):
        if augment : 
            textp=self.aug.augment(textp,n=1)
            textl=self.aug.augment(textl,n=1)

        combined = textp.split(' ')+textl.split(' ')
        split_point = int(np.random.uniform(1,len(combined)))
        textp = ' '.join(combined[:split_point])
        textl = ' '.join(combined[split_point:])
        
        return (self.tokenizer(textp,add_special_tokens = True,
                         truncation=True,return_tensors = 'np',
                         return_attention_mask = True,padding = 'max_length',max_length=self.maxsize),
                self.tokenizer(textl,add_special_tokens = True,
                         truncation=True,return_tensors = 'np',
                         return_attention_mask = True,padding = 'max_length',max_length=self.maxsize))
    
    def convert(self,text,augment=True):
        if augment : 
            text=self.aug.augment(text,n=1)
        return self.tokenizer(text,add_special_tokens = True,
                         truncation=True,return_tensors = 'np',
                         return_attention_mask = True,padding = 'max_length',max_length=self.maxsize)
    
    def get_embeds(self,model,batch_size,device):
        labels = self.convert(self.label_text_files,augment=False)
        points = self.convert(self.point_text_files,augment=False)
        num_labels = labels['input_ids'].shape[0]
        num_points = points['input_ids'].shape[0]
        with torch.no_grad():
            label_embeds = []
            for i in range(0,num_labels,batch_size):
                label_embeds.append(model.get_embed({'input_ids':torch.LongTensor(labels['input_ids'][i:i+batch_size]).to(device),
                                          'attention_mask':torch.LongTensor(labels['attention_mask'][i:i+batch_size]).to(device)}).cpu())
            label_embeds = torch.cat(label_embeds,dim=0)

            point_embeds = []
            for i in range(0,num_points,batch_size):
                point_embeds.append(model.get_embed({'input_ids':torch.LongTensor(points['input_ids'][i:i+batch_size]).to(device),
                                          'attention_mask':torch.LongTensor(points['attention_mask'][i:i+batch_size]).to(device)}).cpu())
            point_embeds = torch.cat(point_embeds,dim=0)
        return point_embeds,label_embeds

    def __len__(self):
        return len(self.mapping[0])

def collate_fn(batch):
    point_ids,label_ids,point_masks,label_masks = zip(*batch)
    return {'input_ids':torch.stack(point_ids,dim=0).long(),'attention_mask':torch.stack(point_masks,dim=0).long()},{'input_ids':torch.stack(label_ids,dim=0).long(),'attention_mask':torch.stack(label_masks,dim=0).long()}


class PrecEvaluator():
    def __init__(self, dataset, device,batch_size):
        self.K = 5
        self.metric = "P"
        self.dataset = dataset
        self.device = device
        self.batch_size = batch_size
        self.filter_mat = dataset.tst_filter_mat
        self.best_score = -9999999

    def __call__(self,model):
        xembs,yembs = self.dataset.get_embeds(model,self.batch_size,self.device)
        torch.cuda.empty_cache()
        es = exact_search({'data': yembs.cpu().numpy(), 'query': xembs.cpu().numpy(), 'K': 100, 'device': self.device})
        score_mat = es.getnns_gpu()
        if self.filter_mat is not None:
            self._filter(score_mat)
        res = self.printacc(score_mat, X_Y=self.dataset.mat_mapping, K=self.K)
        recall = xc_metrics.recall(score_mat, self.dataset.mat_mapping, k=100)*100
        print(f'Recall@100: {"%.2f"%recall[99]}')        
        score = res[str(self.K)][self.metric]
        return score
    
    def _filter(self,score_mat):
        temp = self.filter_mat.tocoo()
        score_mat[temp.row, temp.col] = 0
        del temp
        score_mat = score_mat.tocsr()
        score_mat.eliminate_zeros()
        return score_mat

            
    def printacc(self,score_mat, K = 5, X_Y = None, disp = True):
        if X_Y is None: X_Y = tst_X_Y

        acc = xc_metrics.Metrics(X_Y.tocsr().astype(np.bool), None)
        metrics = np.array(acc.eval(score_mat, K))*100
        df = pd.DataFrame(metrics)

        df.index = ['P', 'nDCG']
        
        df.columns = [str(i+1) for i in range(K)]
        if disp: display(df.round(2))
        return df

In [7]:
class BERTModel(torch.nn.Module):
    def __init__(self,gamma):
        super().__init__()
        self.encoder = SentenceTransformer('msmarco-distilbert-base-v3')
        self.rep_dim = self.encoder.get_sentence_embedding_dimension()
        self.hidden_dim = 1024
        self.target_encoder = copy.deepcopy(self.encoder).requires_grad_(False)
        self.predictor = torch.nn.Sequential(torch.nn.Linear(self.rep_dim,self.hidden_dim),
                                             torch.nn.ReLU(),torch.nn.Linear(self.hidden_dim,self.rep_dim))
        self.gamma = gamma
        
    def forward(self,x,y):
        flip = np.random.binomial(1,0.5)
        if (flip == 0):
            x_embeds = torch.nn.functional.normalize(self.predictor(self.encoder(x)['sentence_embedding']),dim=-1)
            y_embeds = torch.nn.functional.normalize(self.target_encoder(y)['sentence_embedding'],dim=-1).clone().detach()
        else:
            y_embeds = torch.nn.functional.normalize(self.predictor(self.encoder(y)['sentence_embedding']),dim=-1)
            x_embeds = torch.nn.functional.normalize(self.target_encoder(x)['sentence_embedding'],dim=-1).clone().detach()
        scores = x_embeds@y_embeds.T
        positives = torch.diag(scores).sum()
        negatives = scores.sum()-positives
        positives /= scores.shape[0]
        negatives /= ((scores.shape[0]-1)*(scores.shape[0]))
        self.update_target()
        return positives,negatives
    
    def get_embed(self,x):
        return torch.nn.functional.normalize(self.encoder(x)['sentence_embedding'],dim=-1)
        
    
    def update_target(self):
        target_dict = self.target_encoder.state_dict()
        online_dict = self.encoder.state_dict()
        for key in online_dict.keys():
            target_dict[key] = target_dict[key]*self.gamma + online_dict[key]*(1-self.gamma)
        self.target_encoder.load_state_dict(target_dict)

In [12]:
expname = 'WordNetAugCut'
unzipped_dir = "LF-Amazon-131K"
full_data_dir = os.path.join(TOP_DIR, data_dir, unzipped_dir)
trn_X_Y = train_sparse_matrix
tst_X_Y = test_sparse_matrix
device = torch.device('mps')
batch_size = 1024
max_epoch = 500
gamma = 0.995
iteration = 0

model = BERTModel(gamma = gamma).to(device)
results_dir = f'{TOP_DIR}/logs/tensorboard/{expname}_gamma-{gamma}_bs-{batch_size}
writer = SummaryWriter(log_dir=results_dir)


train_dataloader = torch.utils.data.DataLoader(BertDataset(datadir=full_data_dir,mapping=trn_X_Y,split='trn',device=device),batch_size=batch_size,drop_last=True,shuffle=True,collate_fn=collate_fn)
test_dataloader = torch.utils.data.DataLoader(BertDataset(datadir=full_data_dir,mapping=tst_X_Y,split='tst',device=device),batch_size=batch_size,drop_last=True,shuffle=True,collate_fn=collate_fn)

optimizer = AdamW(model.parameters(), lr=1e-4,weight_decay=1e-6)
num_training_steps = max_epoch * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=1000,
    num_training_steps=num_training_steps
)

evaluator = PrecEvaluator(test_dataloader.dataset,device,100)
best_score = float('inf')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/danielfrees/Desktop/causalign/data/LF-Amazon-131K/raw/trn_X.title.txt'

In [29]:
for epoch in range(max_epoch):
    print ("starting epoch %d at iteration %d"%(epoch,iteration))
    for _,(x,y) in enumerate(train_dataloader):
        positives,negatives = model(x,y)
        loss = -positives
        exit()
        if (iteration %100  == 0):
            writer.add_scalar('train/positives',positives,iteration)
            writer.add_scalar('train/negatives',negatives,iteration)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        iteration += 1
    if(epoch %1 == 0):
        positives,negatives,count = 0,0,0
        with torch.no_grad():
            for _,(x,y) in enumerate(test_dataloader):
                positives_,negatives_ = model(x,y)
                positives += positives_
                negatives += negatives_
                count += 1
        score = evaluator.__call__(model)
        writer.add_scalar('val/P@5',score,iteration)
        writer.add_scalar('val/positives',positives/count,iteration)
        writer.add_scalar('val/negatives',negatives/count,iteration)
        if (score < best_score):
            best_score = score
            torch.save(model.state_dict(),os.path.join(results_dir,'checkpoint.pt'))
            print ('saved checkpoint for epoch %d'%epoch)

starting epoch 0 at iteration 0


NameError: name 'train_dataloader' is not defined